# Load and transform the fMRI data for the ABIDE1 Dataset

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import nilearn as nl
from nilearn import datasets
from nilearn import plotting

In [3]:
asd_fmri = nl.datasets.fetch_abide_pcp(data_dir='E:\ASD_cpac',  pipeline='cpac')

asd_fmri.keys()

C:\Users\xavier\anaconda3\envs\exts-ml\lib\site-packages\numpy\lib\npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


dict_keys(['description', 'phenotypic', 'func_preproc'])

In [4]:
func_prep = asd_fmri.func_preproc
print(func_prep[:10])

['E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050003_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050004_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050005_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050006_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050007_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050008_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050010_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050011_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050012_func_preproc.nii.gz', 'E:\\ASD_cpac\\ABIDE_pcp\\cpac\\nofilt_noglobal\\Pitt_0050013_func_preproc.nii.gz']


## Note :
Here I will create multiples datasets with several options, I don't know if I will use everything but since it takes forever to run I will try to capture the most of it.

In [5]:
# import sys
# from nilearn.input_data import NiftiLabelsMasker
# from nilearn import connectome

# # prerequis 
# multiscale = datasets.fetch_atlas_basc_multiscale_2015(resume=True)
# print('---------------------------------------------------------------------------------')
# print()
# print('Atlas ROIs are located at: %s' % multiscale.scale064)
# print()
# print('---------------------------------------------------------------------------------')


# label_masker = NiftiLabelsMasker(labels_img=multiscale.scale064, standardize=True)

# correlation_measure = connectome.ConnectivityMeasure(kind='correlation', vectorize=True) # tangent kind across group

# list_of_matrix = []

# print('Transforming data into matrix...')
# print()
# # iterate through my files, transform them in to a flatten (half) matrix
# for scan in tqdm(range(0, (len(asd_fmri.func_preproc)))):
    
#     if scan == 101:
#         pass
    
#     else:
#         # pathway 
#         fmri = func_prep[scan]

#         # transfom into array
#         fmri_matrix = label_masker.fit_transform(os.path.join(fmri))

#         # transform into matrix 
#         correlation_matrix = correlation_measure.fit_transform([fmri_matrix])

#         # append this matrix to the list
#         list_of_matrix.append(correlation_matrix)

# print()
# print('Data transformed!')

## Create datasets with the correlation/tangent matrix 


In [6]:
import nilearn as nl
from nilearn.input_data import NiftiLabelsMasker
# import warnings filter
from warnings import simplefilter
# ignore all user warnings
simplefilter(action='ignore', category=UserWarning)

# prerequis 
multiscale = datasets.fetch_atlas_basc_multiscale_2015(resume=True)
print('---------------------------------------------------------------------------------')
print()
print('Atlas ROIs are located at: %s' % multiscale.scale064)
print()
print('---------------------------------------------------------------------------------')

ASD = []
control = []
all_subjects = []

label_masker = NiftiLabelsMasker(labels_img=multiscale.scale064, standardize=True).fit()

for subject in tqdm(range(len(func_prep))):
    
    if subject == 101:
        pass 
    else: 
        time_serie = label_masker.transform(os.path.join(func_prep[subject]))
        all_subjects.append(time_serie)
        
        if asd_fmri.phenotypic['DX_GROUP'][subject] == 1:
            ASD.append(time_serie)
        else:
            control.append(time_serie)

print()
print('Time series created.')
print()
print('Data has {0} ASD.'.format(len(ASD)))
print('Data has {0} Control.'.format(len(control)))
print('Data has {0} subjects.'.format(len(all_subjects)))

df_asd = pd.DataFrame(data={'ASD': ASD})
df_asd.to_csv('asd_sub_bygroup_timeseries.csv', index=False, encoding='utf-8')
df_c = pd.DataFrame(data={'Control': control})
df_c.to_csv('control_sub_bygroup_timeseries.csv', index=False, encoding='utf-8')

  0%|                                                                                          | 0/871 [00:00<?, ?it/s]

---------------------------------------------------------------------------------

Atlas ROIs are located at: C:\Users\xavier/nilearn_data\basc_multiscale_2015\template_cambridge_basc_multiscale_nii_sym\template_cambridge_basc_multiscale_sym_scale064.nii.gz

---------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████| 871/871 [1:51:58<00:00,  7.71s/it]



Time series created.

Data has 403 ASD.
Data has 467 Control.
Data has 870 subjects.


### Correlation matrix / tangent matrix

In [7]:
import sys
from nilearn import connectome

print('Save the time series as well, might be usefull')

df = pd.DataFrame(asd_fmri.phenotypic)
# Drop the 101 rows 
df_new = df.drop([101], axis=0)
# create the target
target = df_new.DX_GROUP.values
print('Target shape:', target.shape)

time_series_array = np.asarray(all_subjects)
print('Time serie shape:', time_series_array.shape)

# Save into a npz 
np.savez('fMRI_time_series.npz', matrix=time_series_array, targets=target)

print('Npz file: train saved')

print('---------------------------------------------------------------------------------')
print()
print('Pearson correlation')

correlation_measure = connectome.ConnectivityMeasure(kind='correlation', vectorize=True)

# Transform data into matrix 
matrices_correlation = correlation_measure.fit_transform(all_subjects)

print('Correlation matrix created')
print('Correlation matrix shape:', matrices_correlation.shape)


# Save into a npz 
np.savez('fMRI_matrix.npz', matrix=matrices_correlation, targets=target)

print('Npz file: train saved')


print('---------------------------------------------------------------------------------')
print()
print('Tangent correlation')

tangent_measure = connectome.ConnectivityMeasure(kind='tangent', vectorize=True)

# Transform data into matrix 
matrices_tangent = tangent_measure.fit_transform(all_subjects)

print('Correlation matrix created')
print('Correlation matrix shape:', matrices_tangent.shape)

# Save into a npz 
np.savez('fMRI_tangent_matrix.npz', matrix=matrices_tangent, targets=target)

print('Npz file: train saved')

Save the time series as well, might be usefull
Target shape: (870,)
Time serie shape: (870,)
Npz file: train saved
---------------------------------------------------------------------------------

Pearson correlation
Correlation matrix created
Correlation matrix shape: (870, 2080)
Npz file: train saved
---------------------------------------------------------------------------------

Tangent correlation
Correlation matrix created
Correlation matrix shape: (870, 2080)
Npz file: train saved


# Try the correlations AND the confounds

In [8]:
import nilearn as nl
from nilearn.input_data import NiftiLabelsMasker
# import warnings filter
from warnings import simplefilter
# ignore all user warnings
simplefilter(action='ignore', category=UserWarning)
from nilearn import image
# A "memory" to avoid recomputation
from nilearn._utils.compat import Memory
mem = Memory('nilearn_cache')

# prerequis 
multiscale = datasets.fetch_atlas_basc_multiscale_2015(resume=True)
print('---------------------------------------------------------------------------------')
print()
print('Atlas ROIs are located at: %s' % multiscale.scale064)
print()
print('---------------------------------------------------------------------------------')

ASD_conf = []
control_conf = []
all_subjects_conf = []

label_masker = NiftiLabelsMasker(labels_img=multiscale.scale064, standardize=True).fit()

for subject in tqdm(range(len(func_prep))):
    
    if subject == 101:
        pass 
    else: 
        hv_confounds = mem.cache(image.high_variance_confounds)(func_prep[subject]);
        time_serie = label_masker.transform(os.path.join(func_prep[subject]), confounds=hv_confounds);
        all_subjects_conf.append(time_serie)
        
        if asd_fmri.phenotypic['DX_GROUP'][subject] == 1:
            ASD_conf.append(time_serie)
        else:
            control_conf.append(time_serie)

print()
print('Time series created.')
print()
print('Data has {0} ASD.'.format(len(ASD_conf)))
print('Data has {0} Control.'.format(len(control_conf)))
print('Data has {0} subjects.'.format(len(all_subjects_conf)))

df_asd = pd.DataFrame(data={'ASD': ASD_conf})
df_asd.to_csv('asd_sub_bygroup_timeseries_conf.csv', index=False, encoding='utf-8')
df_c = pd.DataFrame(data={'Control': control_conf})
df_c.to_csv('control_sub_bygroup_timeseries_conf.csv', index=False, encoding='utf-8')

  0%|                                                                                          | 0/871 [00:00<?, ?it/s]

---------------------------------------------------------------------------------

Atlas ROIs are located at: C:\Users\xavier/nilearn_data\basc_multiscale_2015\template_cambridge_basc_multiscale_nii_sym\template_cambridge_basc_multiscale_sym_scale064.nii.gz

---------------------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████| 871/871 [1:52:31<00:00,  7.75s/it]



Time series created.

Data has 403 ASD.
Data has 467 Control.
Data has 870 subjects.


### Correlation / tangent  matrix & confounds

In [9]:
import sys
from nilearn import connectome

print('Save the time series as well, might be usefull')

df = pd.DataFrame(asd_fmri.phenotypic)
# Drop the 101 rows 
df_new = df.drop([101], axis=0)
# create the target
target = df_new.DX_GROUP.values
print('Target shape:', target.shape)

time_series_array = np.asarray(all_subjects_conf)
print('Time serie shape:', time_series_array.shape)

# Save into a npz 
np.savez('fMRI_time_series_confounds.npz', matrix=time_series_array, targets=target)

print('Npz file: train saved')

print('---------------------------------------------------------------------------------')
print()
print('Pearson correlation')

correlation_measure = connectome.ConnectivityMeasure(kind='correlation', vectorize=True)

# Transform data into matrix 
matrices_correlation = correlation_measure.fit_transform(all_subjects_conf)

print('Correlation matrix created')
print('Correlation matrix shape:', matrices_correlation.shape)


# Save into a npz 
np.savez('fMRI_matrix_confounds.npz', matrix=matrices_correlation, targets=target)

print('Npz file: train saved')


print('---------------------------------------------------------------------------------')
print()
print('Tangent correlation')

tangent_measure = connectome.ConnectivityMeasure(kind='tangent', vectorize=True)

# Transform data into matrix 
matrices_tangent = tangent_measure.fit_transform(all_subjects_conf)

print('Correlation matrix created')
print('Correlation matrix shape:', matrices_tangent.shape)

# Save into a npz 
np.savez('fMRI_tangent_matrix_confounds.npz', matrix=matrices_tangent, targets=target)

print('Npz file: train saved')

Save the time series as well, might be usefull
Target shape: (870,)
Time serie shape: (870,)
Npz file: train saved
---------------------------------------------------------------------------------

Pearson correlation
Correlation matrix created
Correlation matrix shape: (870, 2080)
Npz file: train saved
---------------------------------------------------------------------------------

Tangent correlation
Correlation matrix created
Correlation matrix shape: (870, 2080)
Npz file: train saved
